# Student Test Generator

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,PromptTemplate

from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field, validator
from typing import List

In [3]:
llm = ChatOpenAI()

In [4]:
# Define your desired data structure.

class MultipleChoice(BaseModel):
    choice_ans: str = Field(description="multiple choice answer")

class Question(BaseModel):
    question: str = Field(description="the question being asked")
    #answer: str = Field(description="answer to the question")
    MultipleChoices: List[MultipleChoice]
    correct_ans: int = Field(description="the index of the correct answer")

    # You can add custom validation logic easily with Pydantic.
    @validator("question")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


class Questions(BaseModel):
    Questions: List[Question]

# And a query intented to prompt a language model to populate the data structure.
question_query = "Ask me a 3 questions in the specified subject and which would be suitable for students in school level given,\
      and provide 4 multiple choice answers where only 1 is correct. \
        Also provide index of the correct answer."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Questions)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\nsubject: {subject} , school_level: {level}\n",
    input_variables=["query", "subject", "level"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=question_query, subject="English" , level="Grade 12")

output = llm(_input.to_messages())

res = parser.parse(output.content)

In [5]:
for i in res.Questions:
    print(i.question)
    for j in i.MultipleChoices:
        print(j.choice_ans)
    print("Correct Answer: " , int(i.correct_ans) + 1 , ":" , i.MultipleChoices[int(i.correct_ans)].choice_ans)

What is the definition of a metaphor?
A figure of speech that compares two unlike things without using like or as
A type of poem that follows a specific rhyme scheme and rhythm
The repetition of consonant sounds at the beginning of words
A comparison between two unlike things using like or as
Correct Answer:  1 : A figure of speech that compares two unlike things without using like or as
Which of the following is an example of personification?
The stars twinkled in the night sky
The wind whispered through the trees
She sells seashells by the seashore
The waves crashed angrily against the shore
Correct Answer:  2 : The wind whispered through the trees
What is the purpose of a thesis statement in an essay?
To provide a summary of the main points
To introduce the topic of the essay
To present evidence and examples
To express the writer's opinion or argument
Correct Answer:  4 : To express the writer's opinion or argument


In [13]:
import pickle


with open( 'objects.pkl' , 'wb') as file:
    pickle.dump(res, file , pickle.HIGHEST_PROTOCOL)

In [24]:
chain = prompt | llm | parser

In [27]:
res2 = chain.invoke({'query' : question_query , 'subject' : 'science' , 'level' :'Grade 10'})

In [28]:
for i in res2.Questions:
    print(i.question)
    for j in i.MultipleChoices:
        print(j.choice_ans)
    print("Correct Answer: " , int(i.correct_ans) + 1 , ":" , i.MultipleChoices[int(i.correct_ans)].choice_ans)

What is the chemical formula for water?
H2O
CO2
NaCl
C6H12O6
Correct Answer:  1 : H2O
Which gas is responsible for the greenhouse effect?
Oxygen
Carbon dioxide
Nitrogen
Hydrogen
Correct Answer:  2 : Carbon dioxide
What is the unit of electric current?
Volt
Watt
Ampere
Ohm
Correct Answer:  3 : Ampere


In [7]:
Q1 = res.Questions[1]

MCs = []

for k , v in enumerate(Q1.MultipleChoices):
        MCs.append(v.choice_ans)

MCs

['The stars twinkled in the night sky',
 'The wind whispered through the trees',
 'She sells seashells by the seashore',
 'The waves crashed angrily against the shore']

In [14]:
selected = 'Watt'

MCs.index(selected)

ValueError: 'Watt' is not in list

In [20]:
['x' , 'a'] in ["a" , "b" , "c"]

False